In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from zoneinfo import ZoneInfo
from ib_async import *

In [ ]:
util.startLoop()  # Required for notebooks

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=1)

<IB connected to 127.0.0.1:7496 clientId=1>

Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm.nj; hfarm; jfarm; usfuture; cashfarm; usfarm; euhmds; apachmds; cashhmds; fundfarm; ushmds; secdefhk.


In [17]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7496, 186 B sent in 9 messages, 25.6 kB received in 565 messages, session time 2.33 ks.'

In [3]:
# contract = Commodity('XAUUSD', 'SMART', 'USD')
# contract = Future('GC', '202512', 'COMEX')
# contract = Future('GC', '202510', 'COMEX', includeExpired=True)
contract = Future('M-GD', '202610', 'OSE.JPN')

bars = ib.reqHistoricalData(
        contract, endDateTime='', durationStr='1 D',
        barSizeSetting='1 hour', whatToShow='TRADES', useRTH=True)

bars
# df = util.df(bars)
# df.head(5)

[BarData(date=datetime.datetime(2025, 12, 12, 17, 0, tzinfo=zoneinfo.ZoneInfo(key='Japan')), open=21831.0, high=21967.0, low=21821.0, close=21963.0, volume=2798.0, average=21886.9, barCount=2384),
 BarData(date=datetime.datetime(2025, 12, 12, 18, 0, tzinfo=zoneinfo.ZoneInfo(key='Japan')), open=21962.0, high=22027.0, low=21958.0, close=22022.0, volume=2254.0, average=21990.1, barCount=2011),
 BarData(date=datetime.datetime(2025, 12, 12, 19, 0, tzinfo=zoneinfo.ZoneInfo(key='Japan')), open=22023.0, high=22086.0, low=22006.0, close=22082.0, volume=1438.0, average=22048.5, barCount=1352),
 BarData(date=datetime.datetime(2025, 12, 12, 20, 0, tzinfo=zoneinfo.ZoneInfo(key='Japan')), open=22079.0, high=22122.0, low=22063.0, close=22101.0, volume=1833.0, average=22098.8, barCount=1709),
 BarData(date=datetime.datetime(2025, 12, 12, 21, 0, tzinfo=zoneinfo.ZoneInfo(key='Japan')), open=22101.0, high=22110.0, low=22085.0, close=22092.0, volume=302.0, average=22098.5, barCount=291)]

In [ ]:
def get_hist_df(contract, whatToShow):
    bars = ib.reqHistoricalData(
        contract, endDateTime='', durationStr='6 M',
        barSizeSetting='1 hour', whatToShow=whatToShow, useRTH=True)
    return util.df(bars)

In [ ]:
df_a10 = get_hist_df(Future('GC', '202510', 'COMEX', includeExpired=True), 'BID')
df_a12 = get_hist_df(Future('GC', '202512', 'COMEX'), 'TRADES')

df_b = get_hist_df(Commodity('XAUUSD', 'SMART', 'USD'), 'ASK')

In [12]:
tz = ZoneInfo("America/New_York")
# df_a10 = df_a10[(df_a10['date'] >= datetime(2025, 8, 1, 0, 0, 0, tzinfo=tz)) & (df_a10['date'] < datetime(2025, 10, 1, 0, 0, 0, tzinfo=tz))]
# df_a12 = df_a12[df_a12['date'] >= datetime(2025, 10, 1, 0, 0, 0, tzinfo=tz)]
# df_a = pd.concat([df_a10, df_a12], axis=0, ignore_index=True)
df_a = df_a12[df_a12['date'] >= datetime(2025, 9, 16, 0, 0, 0, tzinfo=tz)]

In [13]:
df = pd.merge(df_a, df_b, left_on='date', right_on='date', how='left', suffixes=('_a', '_b'))
df.dropna(inplace=True)
df = df[['date', 'high_a', 'low_a', 'close_a', 'close_b', 'volume_a']]
df['Spread'] = df['close_a'] - df['close_b']

In [ ]:
print(df['Spread'].quantile([0.025, 0.975]))
print(df['Spread'].describe())

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
# plt.scatter(df['date'], df['close_a'], label='a', s=5)
# plt.scatter(df['date'], df['close_b'], label='b', s=5)
plt.scatter(df['date'], df['Spread'], label='Spread', s=5)
plt.axhline(y=10, linestyle='--', linewidth=0.5)
plt.axhline(y=0, linestyle='--', linewidth=0.5)
plt.legend()
plt.show()

In [16]:
df[df['Spread'] < 5]

,date,high_a,low_a,close_a,close_b,volume_a,Spread
271,2025-10-31 16:00:00-04:00,4018.9,4007.6,4013.4,4011.76,3091.0,1.64
